In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip -q install accelerate
!pip -q install bitsandbytes
!pip -q install huggingface_hub

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.5/137.5 MB 9.5 MB/s eta 0:00:00


In [3]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) n
Token is valid (permission: fineGrained).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [4]:
%cd /content/drive/MyDrive/DSD

/content/drive/MyDrive/DSD


In [5]:
import gc
import numpy as np
from pickle import dump
from tqdm.notebook import tqdm
from sklearn.decomposition import FastICA

import torch
from torch import nn
from torch.utils.data import DataLoader, Dataset
from transformers import AutoTokenizer, AutoModel

In [12]:
class CFG:
    model_name = "solidrust/dolphin-2.9.1-llama-3-8b-AWQ"
    max_len = 4
    batch_size = 32
    num_workers = 12
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


In [7]:
loaded_words = []
with open('reddit_l2_w2v_words.txt', 'r') as f:
    for line in f:
        word = line.strip()
        loaded_words.append(word)

In [8]:
class WordsDataset(Dataset):
    def __init__(self, words, tokenizer):
        self.words = words
        self.tokenizer = tokenizer

    def __len__(self):
        return len(self.words)

    def __getitem__(self, idx):
        word = self.words[idx]
        inputs = self.tokenizer(word, padding='max_length', max_length = CFG.max_len, truncation = True)
        return {"input_ids": torch.tensor(inputs.input_ids, dtype=torch.long), "attention_mask": torch.tensor(inputs.attention_mask, dtype=torch.long)}

In [13]:
def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output[0]
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)

class Model(torch.nn.Module):
    def __init__(self, model_name):
        super(Model, self).__init__()
        self.encoder = AutoModel.from_pretrained(model_name, load_in_8bit = True)

    def forward(self, input_ids, attention_mask):
        model_output = self.encoder(input_ids=input_ids, attention_mask=attention_mask)
        sentence_embeddings = mean_pooling(model_output, attention_mask)
        return sentence_embeddings

lm_model = Model(CFG.model_name).to(CFG.device)

tokenizer = AutoTokenizer.from_pretrained(CFG.model_name)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

config.json:   0%|          | 0.00/993 [00:00<?, ?B/s]

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


AttributeError: 'BitsAndBytesConfig' object has no attribute 'get_loading_attributes'

In [ ]:
def get_word_embeddings(words, model):
    model.eval()
    words_dataset = WordsDataset(words, tokenizer)
    words_dataloader = DataLoader(words_dataset, batch_size=CFG.batch_size, shuffle=False, num_workers = CFG.num_workers)
    embeddings = []
    for batch in tqdm(words_dataloader, total = len(words_dataloader)):
        input_ids = torch.squeeze(batch['input_ids'].to(CFG.device), axis = 1)
        attention_mask = torch.squeeze(batch['attention_mask'].to(CFG.device), axis = 1)
        batch_embeddings = model(input_ids, attention_mask).detach().cpu().numpy()
        embeddings.append(batch_embeddings)
    # concatenate the embeddings into a single numpy array
    embeddings = np.concatenate(embeddings, axis=0)
    del words_dataset, words_dataloader
    gc.collect()
    return embeddings

word_embeddings = get_word_embeddings(loaded_words, lm_model)

In [ ]:
word_embeddings_dict = {word: embedding for word, embedding in zip(loaded_words, word_embeddings)}
with open(f"word-embeddings/{CFG.model_name.replace('/','_')}_word_embeddings_reddit-l2.pkl", 'wb') as f:
    dump(word_embeddings_dict, f)